In [1]:
from matplotlib import pyplot as plt
import numpy as np
from tokenizer import tokenizer,token2str,vocab_size
import torch
import torch.nn as nn
from make_model import make_model
from train_and_use import Batch,CrossEntropyLoss,SimpleAdamOptimizer,OptimizerWrapper,train_server_start
from train_and_use import El_greedy_decode,El_text_continue,MC_continue,El_text_continue_stream
from train_and_use import TOGGLE,STOP
from train_and_use import record
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
%matplotlib widget

In [2]:
model = make_model(
    #token是从1开始的，0填充，剩下的用来覆盖全部字节
    vocab_size = vocab_size+1+255,
    embedding_dim = 768,
    key_dim = 128,
    head_number = 12,
    position_information_type = "mask",
    # position_information_type = "sinusoidal",
    # position_information_type = "rotary",
    # position_information_type = "learned",
    enable_affine = True,
    enable_talking_head = True,
    use_diff = False,
    self_attention_block_size = 0,
    feed_forward_dim = 1536,
    enable_layer_norm = True,
    deep = 12,
    dropout_rate = 0.1,
    enable_el_cache = True
).to(device)
model.load_state_dict(torch.load('large_model_13033051.weight',weights_only=True))
model = model.eval()
parameters_n = 0
for p in model.parameters():
    parameters_n += p.data.cpu().numpy().size
print("模型参数量:"+str(parameters_n/1e9)[:4]+"(B)")

模型参数量:0.18(B)


In [3]:
# %%time
# A = ''.join([str(np.random.randint(10)) for _ in range(20)])
# B = ''.join([str(np.random.randint(10)) for _ in range(20)])
# C = ''.join([str(np.random.randint(10)) for _ in range(20)])
# zh_strs = ["   A"+A+"   B"+B+"   C"+C+"   A"+A+"   B"+B+"   C"+C+"   A"+A+"   B"+B+"   C"+C+"   A",
#            "   A"+A+"   B"+B+"   C"+C+"   A"+A+"   B"+B+"   C"+C+"   A"+A+"   B"+B+"   C"+C+"   B",
#            "   A"+A+"   B"+B+"   C"+C+"   A"+A+"   B"+B+"   C"+C+"   A"+A+"   B"+B+"   C"+C+"   C"]
# tokens_batch = [tokenizer(zh_str,5.0) for zh_str in zh_strs]
# tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
# inputs = torch.from_numpy(tokens_batch).to(device).data
# with torch.no_grad():
#     out = El_text_continue(
#         model,inputs,out_length=20,
#         repeat_penalty_value=0.0,
#         temperature=0.01,
#         decay=0.98
#     )
# split = ''
# for o in out:
#     print(token2str(o.cpu().numpy()-255,split=split))

In [4]:
# %%time
# zh_strs = ["大家好，"]*20
# tokens_batch = [tokenizer(zh_str,5.0) for zh_str in zh_strs]
# tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
# inputs = torch.from_numpy(tokens_batch).to(device).data
# with torch.no_grad():
#     out = El_text_continue(
#         model,inputs,out_length=15,
#         repeat_penalty_value=0.0,
#         temperature=1.0,
#         decay=0.98
#     )
# split = ''
# for o in out:
#     print(token2str(o.cpu().numpy()-255,split=split))

In [5]:
# %%time
# zh_strs = ["苏轼，"]*1
# tokens_batch = [tokenizer(zh_str,5.0) for zh_str in zh_strs]
# tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
# inputs = torch.from_numpy(tokens_batch).to(device).data
# with torch.no_grad():
#     out = El_text_continue(
#         model,inputs,out_length=128,
#         repeat_penalty_value=1.0,
#         temperature=0.1,
#         decay=0.99
#     )
# split = ''
# for o in out:
#     print(token2str(o.cpu().numpy()-255,split=split))

In [6]:
# model.load_state_dict(torch.load('large_model_repeat_stable_cold.weight',weights_only=True))

In [9]:
%%time
# zh_strs = [' 1']
zh_strs = ["明月几时有，把酒问青天。不知天上宫阙，"]
# zh_strs = ["花间一壶酒，独酌无相亲。举杯邀明月，"]
# zh_strs = ["月落乌啼霜满天，江枫渔火对愁眠。姑苏"]
tokens_batch = [tokenizer(zh_str,5.0) for zh_str in zh_strs]
tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
inputs = torch.from_numpy(tokens_batch).to(device).data
last_len = -1
print(zh_strs[0],end='')
with torch.no_grad():
    for o in El_text_continue_stream(
        model,inputs,out_length=64,
        repeat_penalty_value=1.0,
        temperature=0.1,
        decay=0.99
    ):
        split = ''
        if o[0,-1] > 255: #确保是完整的字符才可以输出
            print(token2str(o[0][last_len:].cpu().numpy()-255,split=split),end="")
            last_len = -1
        else:
            last_len -= 1
        if token2str(o[0][-5:].cpu().numpy()-255,split=split) == '<end':
            print('>')
            break

明月几时有，把酒问青天。不知天上宫阙，今夕是何年？我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间！转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合总无情，月有CPU times: user 1.06 s, sys: 9.26 ms, total: 1.07 s
Wall time: 1.08 s


In [8]:
# %%time
# #基于树搜索的推理
# zh_str = "我是一个在大规模语料中训练得到的语言模型，现在我要极力控制自己，不要不断的重复说过的内容。请介绍一下喜鹊这种动物。"
# inc = 0
# out_start = 0
# print(zh_str,end='')
# for i in range(32):
#     tokens_batch = tokenizer(zh_str,5.0)
#     tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
#     inputs = torch.from_numpy(tokens_batch).to(device).data
#     with torch.no_grad():
#         o = MC_continue(
#             model,inputs,out_length=16+inc,
#             repeat_penalty_value = 1.0,
#             temperature = 1.0,
#             try_n = 30,
#             acc_n = 30,
#             deep_n = 25,
#             decay = 0.98
#         )
#     new_str = token2str(o.cpu().numpy()-255)
#     if len(zh_str) == len(new_str):
#         inc += 1
#     else:
#         inc = 0
#         out_start = len(zh_str)
#         zh_str = new_str
#         print(zh_str[out_start:],end='')
# print()